<a href="https://colab.research.google.com/github/its-Shivam732/XrayDenoise/blob/main/Xray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Install dependencies
!pip install torch torchvision diffusers accelerate scikit-image pytorch-fid pillow -q

# Import libraries
import os
import torch
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from skimage.util import random_noise
from google.colab import drive
import glob
from PIL import Image
import shutil
import zipfile

# Mount Google Drive for persistent storage
drive.mount('/content/drive', force_remount=True)
output_dir = '/content/drive/MyDrive/xray_denoising_dataset'
os.makedirs(output_dir, exist_ok=True)

# ============================================
# DOWNLOAD DATASET USING KAGGLE API
# ============================================

# Setup Kaggle API
os.makedirs('/root/.kaggle', exist_ok=True)

# Upload your kaggle.json file
print("Please upload your kaggle.json file:")
print("1. Go to https://www.kaggle.com/settings/account")
print("2. Scroll to 'API' section and click 'Create New Token'")
print("3. Upload the downloaded kaggle.json file when prompted below")

from google.colab import files
uploaded = files.upload()  # This will prompt you to upload kaggle.json

# Move kaggle.json to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
dataset_root = '/content/covid_xray/COVID-19_Radiography_Dataset'

if not os.path.exists(dataset_root):
    print("\nDownloading COVID-19 Radiography Dataset from Kaggle...")

    # Download dataset
    !kaggle datasets download -d tawsifurrahman/covid19-radiography-database -p /content/covid_xray

    # Extract the zip file
    print("\nExtracting dataset...")
    zip_path = '/content/covid_xray/covid19-radiography-database.zip'

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/covid_xray')

    # Remove zip file to save space
    os.remove(zip_path)

    print("✓ Dataset downloaded and extracted successfully!")

    # Verify the structure
    if os.path.exists(dataset_root):
        print(f"✓ Dataset verified at {dataset_root}")
    else:
        # Sometimes the extracted folder has a different name
        # List what's actually in the directory
        print("\nContents of /content/covid_xray:")
        !ls -la /content/covid_xray

        # Try to find the actual dataset folder
        for item in os.listdir('/content/covid_xray'):
            item_path = os.path.join('/content/covid_xray', item)
            if os.path.isdir(item_path):
                print(f"\nFound directory: {item_path}")
                # Update dataset_root to the actual path
                dataset_root = item_path
                break
else:
    print(f"✓ Dataset already exists at {dataset_root}")

# Verify dataset directory
if not os.path.exists(dataset_root):
    raise FileNotFoundError(f"Dataset root not found at {dataset_root}")

# Get all image paths from subdirectories
def collect_image_paths(root_dir):
    """Collect all image paths from the dataset structure"""
    image_paths = []
    categories = ['Normal', 'COVID', 'Lung_Opacity', 'Viral Pneumonia']

    for category in categories:
        images_dir = os.path.join(root_dir, category, 'images')
        if os.path.exists(images_dir):
            # Get all PNG and JPEG files
            png_files = glob.glob(os.path.join(images_dir, '*.png'))
            jpg_files = glob.glob(os.path.join(images_dir, '*.jpg'))
            jpeg_files = glob.glob(os.path.join(images_dir, '*.jpeg'))

            category_images = png_files + jpg_files + jpeg_files
            image_paths.extend(category_images)
            print(f"Found {len(category_images)} images in {category}")

    return image_paths

# Continue with rest of your code...

Mounted at /content/drive
Please upload your kaggle.json file:
1. Go to https://www.kaggle.com/settings/account
2. Scroll to 'API' section and click 'Create New Token'
3. Upload the downloaded kaggle.json file when prompted below


Saving kaggle.json to kaggle.json

Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database
License(s): copyright-authors
 97% 753M/778M [00:06<00:00, 212MB/s]
100% 778M/778M [00:06<00:00, 123MB/s]

Extracting dataset...
✓ Dataset downloaded and extracted successfully!
✓ Dataset verified at /content/covid_xray/COVID-19_Radiography_Dataset


In [11]:
# Install dependencies
!pip install torch torchvision diffusers accelerate scikit-image pytorch-fid pillow

# Import libraries
import os
import torch
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from skimage.util import random_noise
from google.colab import drive
import glob
from PIL import Image

# Mount Google Drive for persistent storage
drive.mount('/content/drive', force_remount=True)
output_dir = '/content/drive/MyDrive/xray_denoising_dataset'
os.makedirs(output_dir, exist_ok=True)

# Verify dataset directory
dataset_root = '/content/covid_xray/COVID-19_Radiography_Dataset'
if not os.path.exists(dataset_root):
    raise FileNotFoundError(f"Dataset root not found at {dataset_root}")

# Get all image paths from subdirectories
def collect_image_paths(root_dir):
    """Collect all image paths from the dataset structure"""
    image_paths = []
    categories = ['Normal', 'COVID', 'Lung_Opacity', 'Viral_Pneumonia']

    for category in categories:
        images_dir = os.path.join(root_dir, category, 'images')
        if os.path.exists(images_dir):
            # Get all PNG and JPEG files
            png_files = glob.glob(os.path.join(images_dir, '*.png'))
            jpg_files = glob.glob(os.path.join(images_dir, '*.jpg'))
            jpeg_files = glob.glob(os.path.join(images_dir, '*.jpeg'))

            category_images = png_files + jpg_files + jpeg_files
            image_paths.extend(category_images)
            print(f"Found {len(category_images)} images in {category}")

    return image_paths

# Collect all image paths
all_image_paths = collect_image_paths(dataset_root)
print(f"\nTotal images found: {len(all_image_paths)}")

if len(all_image_paths) == 0:
    raise FileNotFoundError("No images found! Check the directory structure.")

# Limit to 2000 images for faster processing
all_image_paths = all_image_paths[:2000]
print(f"Using {len(all_image_paths)} images for this run")

# Custom Dataset class
class XRayDataset(Dataset):
    def __init__(self, image_paths, transform=None, noise_type='gaussian', noise_level=0.1):
        self.image_paths = image_paths
        self.transform = transform
        self.noise_type = noise_type
        self.noise_level = noise_level

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        try:
            # Load image
            image_path = self.image_paths[idx]
            image = Image.open(image_path).convert('L')  # Grayscale

            # Resize to 256x256
            image = image.resize((256, 256), Image.Resampling.LANCZOS)

            # Convert to numpy array and normalize to [0,1]
            image = np.array(image, dtype=np.float32) / 255.0

            # Add noise
            if self.noise_type == 'gaussian':
                noisy = random_noise(image, mode='gaussian', var=self.noise_level**2, clip=True)
            elif self.noise_type == 'poisson':
                noisy = random_noise(image, mode='poisson', clip=True)
            else:
                noisy = image.copy()

            # Convert to tensors [1, H, W]
            image_tensor = torch.from_numpy(image).unsqueeze(0).float()
            noisy_tensor = torch.from_numpy(noisy.astype(np.float32)).unsqueeze(0).float()

            # Apply normalization to [-1, 1] if transform provided
            if self.transform:
                image_tensor = self.transform(image_tensor)
                noisy_tensor = self.transform(noisy_tensor)

            return {'clean': image_tensor, 'noisy': noisy_tensor}

        except Exception as e:
            print(f"Error processing image {idx} ({self.image_paths[idx]}): {e}")
            # Return a blank image pair as fallback
            blank = torch.zeros(1, 256, 256)
            return {'clean': blank, 'noisy': blank}

# Preprocessing transform (normalization only)
transform = transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]

# Split dataset
train_size = 1500
val_size = min(500, len(all_image_paths) - train_size)

train_paths = all_image_paths[:train_size]
val_paths = all_image_paths[train_size:train_size + val_size]

print(f"\nCreating datasets...")
print(f"Train: {len(train_paths)} images")
print(f"Val: {len(val_paths)} images")

# Create datasets
train_dataset = XRayDataset(train_paths, transform=transform, noise_type='gaussian', noise_level=0.1)
val_dataset = XRayDataset(val_paths, transform=transform, noise_type='gaussian', noise_level=0.1)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

print(f"\nDataLoaders created successfully!")

# Save sample images
def save_samples(loader, num_samples=5, output_dir=output_dir):
    samples_dir = os.path.join(output_dir, 'samples')
    os.makedirs(samples_dir, exist_ok=True)

    try:
        batch = next(iter(loader))
        num_to_save = min(num_samples, len(batch['clean']))

        for i in range(num_to_save):
            # Denormalize from [-1, 1] to [0, 1]
            clean = (batch['clean'][i].squeeze().numpy() * 0.5 + 0.5).clip(0, 1)
            noisy = (batch['noisy'][i].squeeze().numpy() * 0.5 + 0.5).clip(0, 1)

            # Create comparison figure
            plt.figure(figsize=(10, 5))

            plt.subplot(1, 2, 1)
            plt.imshow(clean, cmap='gray', vmin=0, vmax=1)
            plt.title('Clean X-Ray', fontsize=14)
            plt.axis('off')

            plt.subplot(1, 2, 2)
            plt.imshow(noisy, cmap='gray', vmin=0, vmax=1)
            plt.title(f'Noisy (Gaussian σ=0.1)', fontsize=14)
            plt.axis('off')

            plt.tight_layout()
            plt.savefig(os.path.join(samples_dir, f'sample_{i}.png'), dpi=150, bbox_inches='tight')
            plt.close()

        print(f"Saved {num_to_save} sample pairs to {samples_dir}")

    except Exception as e:
        print(f"Error saving samples: {e}")

# Save samples from training set
print("\nGenerating sample images...")
save_samples(train_loader, num_samples=8)

# Save metadata
metadata_path = os.path.join(output_dir, 'metadata.txt')
with open(metadata_path, 'w') as f:
    f.write('X-Ray Denoising Dataset\n')
    f.write('='*50 + '\n\n')
    f.write(f'Source: COVID-19 Radiography Database\n')
    f.write(f'Dataset root: {dataset_root}\n\n')
    f.write(f'Training images: {len(train_paths)}\n')
    f.write(f'Validation images: {len(val_paths)}\n')
    f.write(f'Total images: {len(all_image_paths)}\n\n')
    f.write(f'Image specifications:\n')
    f.write(f'  - Size: 256x256 pixels\n')
    f.write(f'  - Format: Grayscale\n')
    f.write(f'  - Normalization: [-1, 1]\n\n')
    f.write(f'Noise configuration:\n')
    f.write(f'  - Type: Gaussian\n')
    f.write(f'  - Standard deviation (σ): 0.1\n')
    f.write(f'  - Variance: 0.01\n\n')
    f.write(f'DataLoader settings:\n')
    f.write(f'  - Batch size: 16\n')
    f.write(f'  - Num workers: 2\n')
    f.write(f'  - Pin memory: True\n')

print(f"\nMetadata saved to {metadata_path}")

# Test loading a batch
print("\nTesting data loading...")
try:
    test_batch = next(iter(train_loader))
    print(f"✓ Successfully loaded batch!")
    print(f"  - Clean images shape: {test_batch['clean'].shape}")
    print(f"  - Noisy images shape: {test_batch['noisy'].shape}")
    print(f"  - Clean images range: [{test_batch['clean'].min():.3f}, {test_batch['clean'].max():.3f}]")
    print(f"  - Noisy images range: [{test_batch['noisy'].min():.3f}, {test_batch['noisy'].max():.3f}]")
except Exception as e:
    print(f"✗ Error loading batch: {e}")

# Initialize PyTorch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n{'='*50}")
print(f"Setup complete!")
print(f"Device: {device}")
print(f"Output directory: {output_dir}")
print(f"{'='*50}\n")

# Print summary
print("Dataset is ready for training!")
print(f"Access training data with: train_loader")
print(f"Access validation data with: val_loader")
print(f"Check samples at: {os.path.join(output_dir, 'samples')}")

Mounted at /content/drive
Found 10192 images in Normal
Found 3616 images in COVID
Found 6012 images in Lung_Opacity

Total images found: 19820
Using 2000 images for this run

Creating datasets...
Train: 1500 images
Val: 500 images

DataLoaders created successfully!

Generating sample images...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved 8 sample pairs to /content/drive/MyDrive/xray_denoising_dataset/samples

Metadata saved to /content/drive/MyDrive/xray_denoising_dataset/metadata.txt

Testing data loading...
✓ Successfully loaded batch!
  - Clean images shape: torch.Size([16, 1, 256, 256])
  - Noisy images shape: torch.Size([16, 1, 256, 256])
  - Clean images range: [-1.000, 1.000]
  - Noisy images range: [-1.000, 1.000]

Setup complete!
Device: cpu
Output directory: /content/drive/MyDrive/xray_denoising_dataset

Dataset is ready for training!
Access training data with: train_loader
Access validation data with: val_loader
Check samples at: /content/drive/MyDrive/xray_denoising_dataset/samples


In [12]:
# Install additional packages if needed
!pip install scikit-image -q

# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

print("✓ All libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


✓ All libraries imported successfully!
PyTorch version: 2.8.0+cu126
CUDA available: False


In [13]:
# ============================================================================
# CELL 2: DnCNN Model Architecture
# ============================================================================
# Define the DnCNN neural network

class DnCNN(nn.Module):
    """
    DnCNN: Denoising Convolutional Neural Network
    Paper: "Beyond a Gaussian Denoiser: Residual Learning of Deep CNN for Image Denoising"

    Key idea: The network learns to predict the NOISE, not the clean image
    Clean image = Noisy image - Predicted noise
    """
    def __init__(self, channels=1, num_layers=17, num_features=64):
        """
        Args:
            channels: Number of image channels (1 for grayscale, 3 for RGB)
            num_layers: Total number of convolutional layers
            num_features: Number of features in hidden layers
        """
        super(DnCNN, self).__init__()

        layers = []

        # First layer: Conv + ReLU
        layers.append(nn.Conv2d(channels, num_features, kernel_size=3, padding=1, bias=False))
        layers.append(nn.ReLU(inplace=True))

        # Middle layers: Conv + BatchNorm + ReLU
        for _ in range(num_layers - 2):
            layers.append(nn.Conv2d(num_features, num_features, kernel_size=3, padding=1, bias=False))
            layers.append(nn.BatchNorm2d(num_features))
            layers.append(nn.ReLU(inplace=True))

        # Last layer: Conv (to predict residual noise)
        layers.append(nn.Conv2d(num_features, channels, kernel_size=3, padding=1, bias=False))

        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        """
        Forward pass: predicts noise residual

        Args:
            x: Noisy input image [B, C, H, W]
        Returns:
            noise: Predicted noise [B, C, H, W]
        """
        noise = self.dncnn(x)
        return noise


# Test the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_model = DnCNN(channels=1, num_layers=17, num_features=64).to(device)

# Count parameters
num_params = sum(p.numel() for p in test_model.parameters())
print(f"✓ DnCNN model created successfully!")
print(f"  Total parameters: {num_params:,}")
print(f"  Model size: ~{num_params * 4 / 1024 / 1024:.2f} MB")

# Test forward pass
test_input = torch.randn(1, 1, 256, 256).to(device)
test_output = test_model(test_input)
print(f"  Test input shape: {test_input.shape}")
print(f"  Test output shape: {test_output.shape}")
print("  ✓ Forward pass successful!")

del test_model, test_input, test_output
torch.cuda.empty_cache()

✓ DnCNN model created successfully!
  Total parameters: 556,032
  Model size: ~2.12 MB
  Test input shape: torch.Size([1, 1, 256, 256])
  Test output shape: torch.Size([1, 1, 256, 256])
  ✓ Forward pass successful!


✓ Training and validation functions defined!
